In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import *

# Cadastro

In [2]:
cadastro = pd.read_csv('CADASTRAL.txt', sep='\t')

/home/jvcm/tsenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cadastro.head()

,CODIGO_CLIENTE,DATA_NASCIMENTO,SEXO,ESTADO_CIVIL,CIDADE,UF,BAIRRO,CEP,QTD_DEPENDENTES,DATA_CADASTRO,DATA_ADMISSAO,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,RENDA_CJ,OUTRAS_RENDAS,QTD_CARTOES_ADICIONAIS,FLAG_CONTA_BANCO,VALOR_HISTORICO_COMPRAS
0,3,1951/10/21 00:00:00.000000000,F,CA,SANTA ROSA,RS,GLORIA,98900000.0,1.0,1998/10/28 00:00:00.000000000,1990/01/01 00:00:00.000000000,AUT+NOMO,PR,2500,2500,NaN,1,NaN,27093.04
1,4,NaN,M,NaN,SANTO CRISTO,RS,NaN,98960000.0,0.0,NaN,NaN,AUT+NOMO,NaN,NaN,NaN,NaN,0,NaN,34.00
2,5,1966/02/07 00:00:00.000000000,M,CA,SANTO CRISTO,RS,CENTRO,98960000.0,1.0,2007/11/24 00:00:00.000000000,2000/05/01 00:00:00.000000000,ASSALARIADO,PR,1500,0,NaN,0,S,4755.19
3,6,1931/01/04 00:00:00.000000000,F,CA,CANDIDO GODOI,RS,CENTRO,98970000.0,0.0,1998/07/12 00:00:00.000000000,1951/01/01 00:00:00.000000000,APOSENTADO,PR,520,NaN,NaN,0,S,2107.00
4,7,1965/10/07 00:00:00.000000000,M,CA,SANTO CRISTO,RS,INTERIOR,98960000.0,0.0,2007/08/16 00:00:00.000000000,1968/05/01 00:00:00.000000000,AGRICULTOR,PR,300,680,NaN,0,NaN,2523.55


In [4]:
# cadastro = cadastro.iloc[:, [0,9]]
cadastro['DATA_CADASTRO'] = pd.to_datetime(cadastro['DATA_CADASTRO'])
cadastro['CODIGO_CLIENTE'] = cadastro['CODIGO_CLIENTE'].astype(str)

In [5]:
cadastro.head()

,CODIGO_CLIENTE,DATA_NASCIMENTO,SEXO,ESTADO_CIVIL,CIDADE,UF,BAIRRO,CEP,QTD_DEPENDENTES,DATA_CADASTRO,DATA_ADMISSAO,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,RENDA_CJ,OUTRAS_RENDAS,QTD_CARTOES_ADICIONAIS,FLAG_CONTA_BANCO,VALOR_HISTORICO_COMPRAS
0,3,1951/10/21 00:00:00.000000000,F,CA,SANTA ROSA,RS,GLORIA,98900000.0,1.0,1998-10-28,1990/01/01 00:00:00.000000000,AUT+NOMO,PR,2500,2500,NaN,1,NaN,27093.04
1,4,NaN,M,NaN,SANTO CRISTO,RS,NaN,98960000.0,0.0,NaT,NaN,AUT+NOMO,NaN,NaN,NaN,NaN,0,NaN,34.00
2,5,1966/02/07 00:00:00.000000000,M,CA,SANTO CRISTO,RS,CENTRO,98960000.0,1.0,2007-11-24,2000/05/01 00:00:00.000000000,ASSALARIADO,PR,1500,0,NaN,0,S,4755.19
3,6,1931/01/04 00:00:00.000000000,F,CA,CANDIDO GODOI,RS,CENTRO,98970000.0,0.0,1998-07-12,1951/01/01 00:00:00.000000000,APOSENTADO,PR,520,NaN,NaN,0,S,2107.00
4,7,1965/10/07 00:00:00.000000000,M,CA,SANTO CRISTO,RS,INTERIOR,98960000.0,0.0,2007-08-16,1968/05/01 00:00:00.000000000,AGRICULTOR,PR,300,680,NaN,0,NaN,2523.55


In [ ]:
cadastro.info()

Cadastrados em 2017:

In [ ]:
cadastro = cadastro[cadastro['DATA_CADASTRO'] >= pd.Timestamp(datetime.date(2017, 1, 1))]
cadastro = cadastro[cadastro['DATA_CADASTRO'] < pd.Timestamp(datetime.date(2018, 1, 1))]

In [ ]:
len(cadastro)

In [ ]:
cadastro['DATA_LIMITE'] = cadastro['DATA_CADASTRO'].apply(lambda x: x + relativedelta(months=+6))

# Cartão/Carnê (Utilizar para as duas bases)

In [ ]:
df = pd.read_csv('CARTAO.txt', sep='\t')

In [ ]:
df['CODIGO_CLIENTE'] = df['CODIGO_CLIENTE'].astype(str)
df['DATA_VENCIMENTO'] = pd.to_datetime(df['DATA_VENCIMENTO'])
df['DATA_PAGAMENTO'] = pd.to_datetime(df['DATA_PAGAMENTO'])

Inner Join:

In [ ]:
df = pd.merge(df, cadastro, on='CODIGO_CLIENTE', how='inner')

Vencimento em dia de semana:

In [ ]:
df['DATA_VENCIMENTO'] = df['DATA_VENCIMENTO'].apply(lambda x: x + relativedelta(days=+1) if (x.weekday() == 6) else x)
df['DATA_VENCIMENTO'] = df['DATA_VENCIMENTO'].apply(lambda x: x + relativedelta(days=+2) if (x.weekday() == 5) else x)

In [ ]:
df.head()

Flag atraso:

In [ ]:
df['FLAG_ATRASO'] = (pd.isnull(df['DATA_PAGAMENTO']) | (df['DATA_PAGAMENTO'] > df['DATA_VENCIMENTO']))
df['FLAG_ATRASO'] = df['FLAG_ATRASO'].astype(int)

Flag elegível:

In [ ]:
df['FLAG_LIMITE'] = (df['DATA_VENCIMENTO'] <= df['DATA_LIMITE'])
df['FLAG_LIMITE'] = df['FLAG_LIMITE'].astype(int)

In [ ]:
len(df)

In [ ]:
eleg = df.groupby('CODIGO_CLIENTE')['FLAG_ATRASO','FLAG_LIMITE'].max()

In [ ]:
eleg = eleg[eleg['FLAG_LIMITE']>0]

In [ ]:
good = eleg[eleg['FLAG_ATRASO'] == 0]
bad = eleg[eleg['FLAG_ATRASO'] == 1]

In [ ]:
len(good)

In [ ]:
eleg.iloc[:,:-1].to_csv('cartao_classif.csv')

# --------------------------------------------------------------------
# União de Carnê e Cartão

In [6]:
cartao = pd.read_csv('cartao_classif.csv')
carne = pd.read_csv('carne_classif.csv')

In [7]:
clientes = cartao.append(carne)

In [8]:
clientes.head()

,CODIGO_CLIENTE,FLAG_ATRASO
0,10001336,1
1,10002278,1
2,10003940,1
3,10004319,1
4,10010009,0


In [9]:
clientes = clientes.groupby('CODIGO_CLIENTE')['FLAG_ATRASO'].max()

In [10]:
dic = {'CODIGO_CLIENTE': clientes.index, 'CLASS': clientes.values}

In [11]:
df_clientes = pd.DataFrame(data=dic)

In [12]:
df_clientes.CLASS.value_counts()

1    78252
0    18138
Name: CLASS, dtype: int64

In [14]:
df_clientes['CODIGO_CLIENTE'] = df_clientes['CODIGO_CLIENTE'].astype(str)

In [15]:
df_clientes = cadastro.merge(df_clientes, on='CODIGO_CLIENTE', how='inner')

In [16]:
df_clientes.head()

,CODIGO_CLIENTE,DATA_NASCIMENTO,SEXO,ESTADO_CIVIL,CIDADE,UF,BAIRRO,CEP,QTD_DEPENDENTES,DATA_CADASTRO,DATA_ADMISSAO,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,RENDA_CJ,OUTRAS_RENDAS,QTD_CARTOES_ADICIONAIS,FLAG_CONTA_BANCO,VALOR_HISTORICO_COMPRAS,CLASS
0,875,1974/03/03 00:00:00.000000000,F,SO,SANTO CRISTO,RS,PHILIPPSEN,98960000.0,0.0,2017-10-31,2003/02/10 00:00:00.000000000,AUT+NOMO,PR,1000,NaN,NaN,0,NaN,1225.73,1
1,1689,1978/02/20 00:00:00.000000000,M,SO,TOLEDO,PR,JARDIM COOPAGRO,85903370.0,0.0,2017-11-30,NaN,APOSENTADO,AL,1152,NaN,NaN,0,NaN,1045.04,1
2,5140,1956/10/16 00:00:00.000000000,M,SO,SANTA ROSA,RS,CRUZEIRO,98900000.0,0.0,2017-03-13,NaN,APOSENTADO,PR,1900,NaN,NaN,0,NaN,3280.56,1
3,7376,1961/01/22 00:00:00.000000000,M,SE,SANTA ROSA,RS,AUXILIADORA,98796524.0,0.0,2017-11-14,1992/12/07 00:00:00.000000000,ASSALARIADO,PR,2182,NaN,NaN,0,S,1337.71,1
4,9042,1968/08/22 00:00:00.000000000,F,CA,SANTA ROSA,RS,CENTRAL,98787404.0,0.0,2017-02-20,2004/07/16 00:00:00.000000000,ASSALARIADO,PR,3335,1800,NaN,0,NaN,5987.88,1


In [18]:
df_clientes.CLASS.value_counts()

1    78252
0    18138
Name: CLASS, dtype: int64

In [19]:
df_clientes.to_csv('TABELAO_FINAL.csv', index = False)